# Users Not Recommend

- UsersNotRecommend toma como parámetro un año particular, y retorna el **top 3** de  de juegos **MENOS** recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)


- Para evitar consultas lentas, crearemos una tabla auxiliar que será agrupada por año y jugador, donde agruparemos los generos en un str y sumaremos el tiempo jugado del usuario.

In [1]:
import pandas as pd
import os

path_reviews = os.path.join('..','data','clear','user_reviews.csv.gz')
path_steam_games = os.path.join('..','data','clear','steam_games.csv.gz')


## Carga de data

In [2]:
## Cargamos la data necesaria para solucionar este problema.
steam_colums = ['id','app_name']
steam_games = pd.read_csv(path_steam_games, usecols = steam_colums)

## renombraremos la columna id por steam_id
steam_games.rename(columns={'id':'steam_id'}, inplace=True)


reviews_cols = ['item_id', 'sentiment_analysis', 'recommend', 'year_posted']
user_reviews = pd.read_csv(path_reviews, compression='gzip',usecols=reviews_cols,lineterminator='\n')


## Creación de tabla consulta

- En esta sección combinaremos user_reviews con user_items, por medio de sus ids.

- Vamos a hacer un join con steam_games y user_reviews,
dado que necesitamos los juego mas recomendados por usuarios, necesitamos los usuarios que hicieron almenos un comentario. es por esto que utilizaremos un inner join para no tener comentarios de usuarios desconocidos y usuarios que no realizaron comentarios

In [3]:
user_reviews.head(4)

,item_id,year_posted,recommend,sentiment_analysis
0,1250,2011,1,2
1,22200,2011,1,1
2,43110,2011,1,2
3,251610,2014,1,2


In [4]:
mask = (user_reviews['recommend'] == 0 ) & (user_reviews['sentiment_analysis'].isin([0]))

user_reviews__not_recommend = user_reviews[mask]

In [5]:
steam_reviews_not_recommend = steam_games.merge(user_reviews__not_recommend,
                              left_on='steam_id', 
                              right_on= 'item_id',
                              how='inner').drop(columns=['steam_id','item_id'])

In [6]:
steam_reviews_not_recommend.head(5)

,app_name,year_posted,recommend,sentiment_analysis
0,The ship: murder party,2014,0,0
1,Defcon,2014,0,0
2,Garry's mod,2014,0,0
3,Garry's mod,2014,0,0
4,Garry's mod,2014,0,0


In [7]:
tabla_endpoint_4 = steam_reviews_not_recommend[['year_posted','app_name','recommend']].groupby(['year_posted','app_name']).count()

In [8]:
tabla_endpoint_4

recommend
year_posted app_name                                        
2011        And yet it moves                               2
            Men of war: vietnam                            1
2012        Call of duty®: black ops ii                    1
            Red faction®: armageddon™                      1
            Resident evil: operation raccoon city          1
...                                                      ...
2015        Xcom: enemy unknown                            1
            Yet another zombie defense                     1
            Zombi                                          1
            Zombie army trilogy                            1
            Zombie zoeds                                   1

[637 rows x 1 columns]

In [9]:
path_endpoint_4 = os.path.join('..','data','clear','04_users_recommend_not_recommend.csv.gz')

tabla_endpoint_4.to_csv(path_endpoint_4, compression='gzip')

## Lectura de tabla consulta

In [10]:
path_endpoint_4 = os.path.join('..','data','clear','04_users_recommend_not_recommend.csv.gz')
table_4 = pd.read_csv(path_endpoint_4)

In [19]:
table_4['year_posted'].unique()

array([2011, 2012, 2013, 2014, 2015])

## Construcción de la consulta
- A continuación vamos a realizar las consultas necesarias para encontrar el **top 3** de los juegosm ménos recomendados.
Para responder este problema vamos a ejemplificar la consulta inspecciónar con un año particular, en este caso **2012**, luego vamos a construir las consultas para este año particular

In [21]:
year = 2015


In [22]:
table_4

,year_posted,app_name,recommend
0,2011,And yet it moves,2
1,2011,Men of war: vietnam,1
2,2012,Call of duty®: black ops ii,1
3,2012,Red faction®: armageddon™,1
4,2012,Resident evil: operation raccoon city,1
...,...,...,...
632,2015,Xcom: enemy unknown,1
633,2015,Yet another zombie defense,1
634,2015,Zombi,1
635,2015,Zombie army trilogy,1


In [14]:
## Retornaremos el top 3 de juegos mas recomendados para el año 2011

top_3 = table_4[['app_name','recommend']][table_4['year_posted']==year].groupby('app_name').count().nlargest(3,'recommend').reset_index()


In [24]:
top_3

,app_name,recommend
0,100% orange juice,1
1,12 labours of hercules ii: the cretan bull,1
2,8bitmmo,1


In [16]:
# Crea el diccionario con los puestos y los nombres de las aplicaciones
diccionario_resultado = {f'Puesto{i+1}': juego for i, juego in enumerate(top_3['app_name'])}

In [25]:
[diccionario_resultado]

[{'Puesto1': 'And yet it moves', 'Puesto2': 'Men of war: vietnam'}]

## Funcion UsersNotRecommend

In [32]:
def user_reviews_not_recommend(year: int):
  path_endpoint_4 = os.path.join('..','data','clear','04_users_recommend_not_recommend.csv.gz')
  table_4 = pd.read_csv(path_endpoint_4)
  
  top_3 = table_4[['app_name','recommend']][table_4['year_posted']==year].groupby('app_name').sum().nlargest(3,'recommend').reset_index()
  
  diccionario_resultado = {f'Puesto{i+1}': juego for i, juego in enumerate(top_3['app_name'])}
  
  return [diccionario_resultado]
  

In [37]:
user_reviews_not_recommend(2012)

[{'Puesto1': 'Call of duty®: black ops ii',
  'Puesto2': 'Red faction®: armageddon™',
  'Puesto3': 'Resident evil: operation raccoon city'}]